In [25]:
import xarray as xr
import pandas as pd
import numpy as np
from ntv_numpy import Xdataset
import ntv_pandas as npd
import cbor2


In [70]:
xds = xr.Dataset(
    {
        "var": (['a', 'b', 'c', 'd', 'e'], np.arange(100000).reshape(10, 10, 10, 10, 10))
    },
    coords={
        "a": np.arange(10),
        "b": np.arange(10),
        "c": np.arange(10),
        "d": np.arange(10),
        "e": np.arange(10),
    }
)
xnd = Xdataset.from_xarray(xds)
jsn_txt = xnd.to_json(notype=[True]*len(xnd), encoded=True)
jsn = xnd.to_json(notype=[True]*len(xnd))
#jsn = xnd.to_json(notype=[True]*len(xnd), format=['complete']*len(xnd))
jsb = cbor2.dumps(jsn)

In [72]:
df = xnd.to_dataframe(json_name=False, info=False).reset_index().sample(frac=1)
csv = df.to_csv()
df.to_csv('test1.csv')
print(len(jsb)/1000, len(jsn_txt)/1000, len(csv)/1000) # parquet 1811 k

368.763 689.172 2277.796


In [73]:
xnd = Xdataset.from_dataframe(df)
jsn_txt = xnd.to_json(notype=[True]*len(xnd), encoded=True)
jsn = xnd.to_json(notype=[True]*len(xnd))
#jsn = xnd.to_json(notype=[True]*len(xnd), format=['complete']*len(xnd))
jsb = cbor2.dumps(jsn)
print(len(jsb)/1000, len(jsn_txt)/1000)

368.763 689.172


entiers : 368.763 689.172 2277.796 parquet 1811 (1985 avec sample)
+ string ('e' 6 car) : 368.823 689.242 2777.796 parquet 1812
+ random 'a' -> 'd' : 369.143 689.928 9637.796 parquet 1813
+ random 'e' 12 car : 369.203 689.994 10297.796 parquet 1813
+ sample  : 369.203 689.994 10297.796 parquet 1987
+ random 'a' -> 'e' 12 car : 369.363 689.822 7777.796 parquet 1986

In [65]:
np.random.rand(4, 3)
xds = xr.Dataset(
    {
        "var": (['a', 'b', 'c', 'd', 'e'], np.arange(100000).reshape(10, 10, 10, 10, 10))
    },
    coords={
        "a": np.random.rand(10),
        "b": np.random.rand(10),
        "c": np.random.rand(10),
        "d": np.random.rand(10),
        "e": np.array(['azertyazerty', 'zertyuzertyu', 'rtyuiortyuio', 'uiopqsuiopqs', 'qsdfghqsdfgh', 
                       'sdfghjsdfghj', 'dfghjkdfghjk', 'ghjklmghjklm', 'wxcvbnwxcvbn', 'hjklmwhjklmw']),
    }
)
xnd = Xdataset.from_xarray(xds)
jsn_txt = xnd.to_json(notype=[True]*len(xnd), encoded=True)
jsn = xnd.to_json(notype=[True]*len(xnd))
#jsn = xnd.to_json(notype=[True]*len(xnd), format=['complete']*len(xnd))
jsb = cbor2.dumps(jsn)

In [66]:
df = xnd.to_dataframe(json_name=False).reset_index()
csv = df.to_csv()
df.to_csv('test1.csv')
print(len(jsb)/1000, len(jsn_txt)/1000, len(csv)/1000) # parquet 1811 k

369.203 689.994 10297.796


In [68]:
np.random.rand(4, 3)
xds = xr.Dataset(
    {
        "var": (['a', 'b', 'c', 'd', 'e'], np.arange(100000).reshape(10, 10, 10, 10, 10))
    },
    coords={
        "a": np.array(['sdfghjsdfghj', 'dfghjkdfghjk', 'azertyazerty', 'zertyuzertyu', 'rtyuiortyuio', 
                       'uiopqsuiopqs', 'qsdfghqsdfgh', 'ghjklmghjklm', 'wxcvbnwxcvbn', 'hjklmwhjklmw']),
        "b": np.array(['uiopqsuiopqs', 'qsdfghqsdfgh', 'azertyazerty', 'zertyuzertyu', 'rtyuiortyuio',
                       'sdfghjsdfghj', 'dfghjkdfghjk', 'wxcvbnwxcvbn', 'ghjklmghjklm', 'hjklmwhjklmw']),
        "c": np.array(['dfghjkdfghjk', 'ghjklmghjklm', 'azertyazerty', 'uiopqsuiopqs', 'qsdfghqsdfgh', 
                       'sdfghjsdfghj', 'wxcvbnwxcvbn', 'hjklmwhjklmw', 'zertyuzertyu', 'rtyuiortyuio']),
        "d": np.array(['azertyazerty', 'zertyuzertyu', 'rtyuiortyuio', 'uiopqsuiopqs', 'qsdfghqsdfgh', 
                       'sdfghjsdfghj', 'dfghjkdfghjk', 'ghjklmghjklm', 'wxcvbnwxcvbn', 'hjklmwhjklmw']),
        "e": np.array(['qsdfghqsdfgh', 'sdfghjsdfghj', 'dfghjkdfghjk', 'ghjklmghjklm', 'wxcvbnwxcvbn',
                       'hjklmwhjklmw', 'azertyazerty', 'zertyuzertyu', 'rtyuiortyuio', 'uiopqsuiopqs'])
    }
)
xnd = Xdataset.from_xarray(xds)
jsn_txt = xnd.to_json(notype=[True]*len(xnd), encoded=True)
jsn = xnd.to_json(notype=[True]*len(xnd))
#jsn = xnd.to_json(notype=[True]*len(xnd), format=['complete']*len(xnd))
jsb = cbor2.dumps(jsn)

In [69]:
df = xnd.to_dataframe(json_name=False).reset_index().sample(frac=1)
csv = df.to_csv()
df.to_csv('test1.csv')
print(len(jsb)/1000, len(jsn_txt)/1000, len(csv)/1000) # parquet 1811 k

369.363 689.822 7777.796
